 --- 
# UCI - Default from Credit Card Clients
---

# Dataset presentation

This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005.

It can be found here:
https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

## Variables
There are 25 variables:

* ID: ID of each client
* LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
* SEX: Gender (1=male, 2=female)
* EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
* MARRIAGE: Marital status (1=married, 2=single, 3=others)
* AGE: Age in years
* PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
* PAY_2: Repayment status in August, 2005 (scale same as above)
* PAY_3: Repayment status in July, 2005 (scale same as above)
* PAY_4: Repayment status in June, 2005 (scale same as above)
* PAY_5: Repayment status in May, 2005 (scale same as above)
* PAY_6: Repayment status in April, 2005 (scale same as above)
* BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
* BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
* BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
* BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
* BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
* BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
* PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
* PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
* PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
* PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
* PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
* PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
* default.payment.next.month: Default payment (1=yes, 0=no)


In [1]:
%load_ext autoreload
%autoreload 2

# Useful imports

## Packages

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option('display.max_columns', 120)

In [4]:
sns.set_style("darkgrid")

## Data Preparation imports

from sklearn.preprocessing import StandardScaler

## Model imports

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [6]:
import xgboost as xgb

In [7]:
import lightgbm as lgb

In [8]:
# import catboost

## Preproecssing imports

In [9]:
from sklearn.pipeline import Pipeline

## Metrics imports

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [11]:
import os
os.environ['PATH'].split(';')

['C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\Library\\mingw-w64\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\Library\\usr\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\Library\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\Scripts',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\condabin',
 'C:\\Program Files\\Docker\\Docker\\Resources\\bin',
 'C:\\ProgramData\\Oracle\\Java\\javapath',
 'C:\\WINDOWS\\system32',
 'C:\\WINDOWS',
 'C:\\WINDOWS\\System32\\Wbem',
 'C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0',
 'C:\\Program Files\\PuTTY',
 'C:\\Program Files\\Intel\\WiFi\\bin',
 'C:\\Program Files\\Common Files\\Intel\\WirelessCommon',
 'C:\\Program Files\\Git LFS',
 'C:\\Program Files\\Pandoc',
 'C:\\Program Files\\MiKTeX 2.9\\miktex\\bin\\x64',
 'C:\\Users\\twang\\AppData\\Local\\Conti

# Importing our Data easily!

In [12]:
path = 'UCI_Credit_Card.csv'

In [13]:
from dataprep.load import load_raw_data
from dataprep.load import load_data
from dataprep.load import load_data_xy


df = load_data(path)
X_raw, y_raw = load_data_xy(path)

# Defining our train/test sets

## Splitting our Df into test/train/val

In [14]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, 
                                    test_size=6000, 
                                    stratify=df.default, 
                                    random_state=42)
df_train, df_test = train_test_split(df_train, 
                                     test_size=6000, 
                                     stratify=df_train.default, 
                                     random_state=42)

## Splitting into (X,y)

In [15]:
from dataprep.load import df2xy

X_train, y_train = df2xy(df_train, 'default')
X_test, y_test = df2xy(df_test, 'default')
X_val, y_val = df2xy(df_val, 'default')

# Protocol for pipelined workflow

Puis on se doit de définir les dictionnaires / classes que l'on va utiliser pour stocker les modèles, leurs scores selon toutes les métriques

Ainsi, on pourra les capitalizer facilement

# Elementary Pipelines

Elementary Pipelines are Pipelines that only do a little processing, such as adding / removing a single feature

## Elementary Transformers from Packages

In [16]:
from sklearn.preprocessing import StandardScaler

## Custom Transformers

In [17]:
# Gather data by age group
from dataprep.pipelines import AgeBinAdder
# Gender x Marriage new category
from dataprep.pipelines import GenderXMarriageAdder
# Gender x AgeBin new category
from dataprep.pipelines import GenderXAgeBinAdder
# Predict next month's bill statement
from dataprep.pipelines import NextBillAdder
# Get_dummies to Df
from dataprep.pipelines import CategoricalWarrior
# Drop a column
from dataprep.pipelines import ColumnDropper


# Prediction Classifiers

Let's define the different classifiers that we will be using

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


In [19]:
dico_models = {}

In [20]:
dico_models['log'] = log_clf = LogisticRegression(C=0.1, 
                             solver='liblinear',
                             penalty='l2',
                             class_weight='balanced', 
                             random_state=42, 
                             n_jobs=-1)
dico_models['svm'] =  SVC(gamma='auto', C=1, class_weight='balanced')
dico_models['tree'] = DecisionTreeClassifier(criterion='entropy', 
                                  random_state=42, 
                                 max_leaf_nodes=5,
                                 class_weight='balanced')
dico_models['forest'] = RandomForestClassifier(n_estimators=500, 
                                 max_leaf_nodes=10, 
                                 n_jobs=-1, 
                                 class_weight='balanced',
                                random_state=42)

dico_models['adaboost'] = AdaBoostClassifier()

In [21]:
dico_models['xgboost'] = xgb.XGBClassifier(scale_pos_weight=5)

# Prediction Pipelines

Prediction Pipelines consists of the following architecture:  
  [Complex Pipeline, Classifier]

In [22]:
from collections import defaultdict

dico_pipelined_models_num = defaultdict(dict)
dico_pipelined_models_cat = defaultdict(dict)

## Defining which preprocessing pipelines we should use, for each model

In [23]:
from sklearn.pipeline import make_pipeline

In [24]:
dico_appropriate_preproc = defaultdict(list)

### Logistic Regression

In [25]:
dico_appropriate_preproc['log'] = []

dico_appropriate_preproc['log'].append( make_pipeline(StandardScaler()) )
dico_appropriate_preproc['log'].append( make_pipeline(NextBillAdder(), 
                                                      StandardScaler()) )
dico_appropriate_preproc['log'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) )
dico_appropriate_preproc['log'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']),
                                                      StandardScaler()) )
dico_appropriate_preproc['log'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) ) 
dico_appropriate_preproc['log'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']), 
                                                      StandardScaler()) )

### SVM

In [26]:
dico_appropriate_preproc['svm'] = []

dico_appropriate_preproc['svm'].append( make_pipeline(StandardScaler()) )
dico_appropriate_preproc['svm'].append( make_pipeline(NextBillAdder(), 
                                                      StandardScaler()) )
dico_appropriate_preproc['svm'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']), 
                                                      StandardScaler()) )

### Decision Tree

In [27]:
dico_appropriate_preproc['tree'] = []

dico_appropriate_preproc['tree'].append( make_pipeline(StandardScaler()) )
dico_appropriate_preproc['tree'].append( make_pipeline(NextBillAdder(), 
                                                      StandardScaler()) )
dico_appropriate_preproc['tree'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) )
dico_appropriate_preproc['tree'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) ) 
dico_appropriate_preproc['tree'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']), 
                                                      StandardScaler()) )

### Random Forest

In [28]:
dico_appropriate_preproc['forest'] = []

dico_appropriate_preproc['forest'].append( make_pipeline(StandardScaler()) )
dico_appropriate_preproc['forest'].append( make_pipeline(NextBillAdder(), 
                                                      StandardScaler()) )
dico_appropriate_preproc['forest'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) )
dico_appropriate_preproc['forest'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) ) 
dico_appropriate_preproc['forest'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']), 
                                                      StandardScaler()) )

### Adaboost

It overfits the majority class 'default' too much, let's stop experimenting on it!

### XGBoost

In [29]:
dico_appropriate_preproc['xgboost'] = []

dico_appropriate_preproc['xgboost'].append( make_pipeline(StandardScaler()) )
dico_appropriate_preproc['xgboost'].append( make_pipeline(NextBillAdder(), 
                                                      StandardScaler()) )
dico_appropriate_preproc['xgboost'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) )
dico_appropriate_preproc['xgboost'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']) ) ) 
dico_appropriate_preproc['xgboost'].append( make_pipeline(AgeBinAdder(), 
                                                      GenderXAgeBinAdder(), 
                                                      GenderXMarriageAdder(), 
                                                      NextBillAdder(), 
                                                      ColumnDropper('age'), 
                                                      CategoricalWarrior(['gender', 'education', 'marriage']), 
                                                      StandardScaler()) )

### CatBoost

## Defining all the Prediction Pipelines to train

In [30]:
dico_pipelined_models_num = defaultdict(dict)

for (name, model) in dico_models.items():
#     print(name)
    for (i, complex_pipe) in enumerate(dico_appropriate_preproc[name]):
#         print(i)
        dico_pipelined_models_num[name][i] = Pipeline([
            ('preproc', complex_pipe),
            ('classifier', model)
        ])

In [31]:
dico_pipelined_models_num

defaultdict(dict, {'log': {0: Pipeline(memory=None,
                       steps=[('preproc',
                               Pipeline(memory=None,
                                        steps=[('standardscaler',
                                                StandardScaler(copy=True, with_mean=True,
                                                               with_std=True))],
                                        verbose=False)),
                              ('classifier',
                               LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                                                  fit_intercept=True, intercept_scaling=1,
                                                  l1_ratio=None, max_iter=100,
                                                  multi_class='warn', n_jobs=-1, penalty='l2',
                                                  random_state=42, solver='liblinear',
                                                  tol=0.0001, verbos

# Training

## Training all the Prediction Pipelines

In [32]:
dico_classification_reports = defaultdict(dict)
dico_confusion_matrices = defaultdict(dict)

In [33]:
for (name, model) in dico_models.items():
    print(name.upper())
    print('-------------')
    for (i, complex_pipe) in enumerate(dico_pipelined_models_num[name]):
#         print('!!! POURQUOI CA MARCHE PAS !!!')
        pipe = dico_pipelined_models_num[name][i]
        pipe.fit(X_train, np.ravel(y_train))
        print('Pipe {}, {}'.format(i, name.capitalize()))
        print('----------------')
        
        y_pred = pipe.predict(X_test)
        dico_classification_reports[name][i] = classification_report(y_test, y_pred)
        dico_confusion_matrices[name][i] = confusion_matrix(y_test, y_pred)
        print(dico_classification_reports[name][i])
        #print(dico_confusion_matrices[i][name])

#         print('!!! POURQUOI CA MARCHE PAS !!!')'

LOG
-------------


C:\Users\twang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipe 0, Log
----------------
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4673
           1       0.48      0.57      0.52      1327

    accuracy                           0.77      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.78      0.77      0.78      6000



C:\Users\twang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipe 1, Log
----------------
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4673
           1       0.48      0.57      0.52      1327

    accuracy                           0.77      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.78      0.77      0.78      6000



...

Pipe 3, Log
----------------
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4673
           1       0.49      0.57      0.53      1327

    accuracy                           0.77      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.79      0.77      0.78      6000



C:\Users\twang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipe 4, Log
----------------
              precision    recall  f1-score   support

           0       0.81      0.78      0.80      4673
           1       0.33      0.37      0.34      1327

    accuracy                           0.69      6000
   macro avg       0.57      0.58      0.57      6000
weighted avg       0.71      0.69      0.70      6000



C:\Users\twang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipe 5, Log
----------------
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      4673
           1       0.49      0.57      0.53      1327

    accuracy                           0.77      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.79      0.77      0.78      6000

SVM
-------------
Pipe 0, Svm
----------------
              precision    recall  f1-score   support

           0       0.87      0.80      0.84      4673
           1       0.46      0.59      0.52      1327

    accuracy                           0.76      6000
   macro avg       0.67      0.70      0.68      6000
weighted avg       0.78      0.76      0.77      6000

Pipe 1, Svm
----------------
              precision    recall  f1-score   support

           0       0.87      0.80      0.84      4673
           1       0.46      0.59      0.52      1327

    accuracy                           0.76      6000
   macro avg       0.67

## Our takeaways

### Per model

* Logistic regression:  
We have a great recall of our default class, using all our engineered features, as well as keeping our 'age' feature

* SVM:  
Adding our previously engineered features leads to no change

* Decision Tree:  
All the different preprocessing pipelines lead to similar results

* Random Forest:  
All the different preprocessing pipelines lead to similar results

* XGBoost:  
All the different preprocessing pipelines lead to similar results

In conclusion, it seems that all our new features aren't too useful, excepted for the Logistic Regression model

### Ranked by performance on test set

Let's have a look at the best (preprocessing, model) couples:   
(best = best recall score on the default class, evaluated on the Test set)

In [34]:
def show_classification_report(model_name, pipe_number):
    print('{}, Pipe {}'.format(model_name.capitalize(), pipe_number))
    print('----------------')
    print(dico_classification_reports[model_name][pipe_number])
    print('------------------------------------------------------')

In [35]:
show_classification_report('log', 2)
show_classification_report('svm', 0)
show_classification_report('forest', 3)
show_classification_report('xgboost', 1)

Log, Pipe 2
----------------
              precision    recall  f1-score   support

           0       0.86      0.51      0.64      4673
           1       0.29      0.71      0.41      1327

    accuracy                           0.56      6000
   macro avg       0.58      0.61      0.53      6000
weighted avg       0.74      0.56      0.59      6000

------------------------------------------------------
Svm, Pipe 0
----------------
              precision    recall  f1-score   support

           0       0.87      0.80      0.84      4673
           1       0.46      0.59      0.52      1327

    accuracy                           0.76      6000
   macro avg       0.67      0.70      0.68      6000
weighted avg       0.78      0.76      0.77      6000

------------------------------------------------------
Forest, Pipe 3
----------------
              precision    recall  f1-score   support

           0       0.88      0.80      0.84      4673
           1       0.46      0.62    

Meilleurs modèles (en comparant selon le recall des default):
* (XGBoost, 1): avec la prédiction du Next Bill
* (Logistic Regression, 2):, avec toutes les features engineerées et sans Scaling (!)
* (Random Forest, 3): (peu importe le type de preprocessing
* (SVM, 0): (scalé absolument, avec ou sans Next Bill ^^ )

# Fine-tuning our best models

# TO DO Next